## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

The core objective was to design, build, and validate a secure, context-aware AI Medical Assistant utilizing a Retrieval-Augmented Generation (RAG) architecture.

Key Focus Areas:
Grounded Response Generation: Ensure the Large Language Model (LLM) only generates responses based on verified, external knowledge sources (Merck Manuals) to eliminate hallucinations and ensure clinical accuracy.

Contextual Precision: Implement efficient vectorization and retrieval mechanisms to pull the most relevant protocols and diagnostic information for a given complex patient query.

End-to-End System Deployment: Demonstrate proficiency in the full NLP lifecycle, from text preprocessing and embedding to orchestrating the RAG pipeline using industry-standard libraries like LangChain and Llama-cpp.

The successful outcome is a robust prototype capable of providing evidence-based, low-latency, and highly contextual answers to complex medical queries, thereby streamlining clinical efficiency and improving patient protocol adherence.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 180.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 150.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 174.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 241.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 188.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.3.2 which is incompatible.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.3.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
xarray 2025.7.

In [ ]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 pypdf langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
xarray 2025.7.1 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
xarray 2025.7.1 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
xarray 2025.7.1 requires pandas>=2.2, but you have pandas 1.5.3 which is incompatible.
datasets 4.0.0 requires huggingface-hub>=0.24.0, but you have huggingface-hub 0.23.2 which is incompatible.
arviz 0.22.0 requires numpy>=1.26.0, but you have numpy 1.25.2 which is incompatible.
arviz 0.22.0 requires pandas>=2.1.0, but you have pandas 1.5.3 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.25.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0,

In [ ]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_S.gguf"
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q5_K_S.gguf:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=38,
    n_ctx=2048
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Response

In [ ]:
def model_response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = lcpp_llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
response = model_response(user_prompt)
print(response)



Sepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care setting:

1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.
2. Resusc


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response = model_response(user_prompt)
print(response)

Llama.generate: prefix-match hit




Appendicitis is a medical condition characterized by inflammation of the appendix, a small pouch that extends from the large intestine. The following are common symptoms of appendicitis:

1. Abdominal pain: The pain typically starts around the navel area and then shifts to the lower right side of the abdomen. It may be constant or come and go, and it can range from mild to severe.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain and discomfort.
3. Nausea and vomiting:


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response = model_response(user_prompt)
print(response)

Llama.generate: prefix-match hit




Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body, such as the beard area or eyebrows.

The exact cause of alopecia areata is not known, but it's believed to be related to a combination of genetic and environmental factors. Some possible triggers include stress, illness, trauma, and certain medications.

There is no cure for alopec


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response = model_response(user_prompt)
print(response)

Llama.generate: prefix-match hit




A person who has sustained a physical injury to the brain, also known as a traumatic brain injury (TBI), may require various treatments depending on the severity and location of the injury. Here are some common treatments recommended for individuals with TBIs:

1. Emergency care: In case of a severe TBI, emergency care is essential to prevent further damage or complications. This may include surgery to remove hematomas or other clots, intubation to help the person breathe, and administration of medications to manage swelling or prevent seizures.
2. Rehabilitation: Rehabil


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_prompt = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response = model_response(user_prompt)
print(response)

Llama.generate: prefix-match hit




A fractured leg is a serious injury that requires prompt medical attention. Here are some necessary precautions and treatment steps for a person who has sustained a leg fracture while hiking:

1. Assess the situation: If you or someone in your group has sustained a leg fracture, assess the situation to determine the severity of the injury. Check for signs of shock, such as pale skin, rapid heartbeat, and shallow breathing. If there are any signs of shock, try to keep the person warm and calm while waiting for medical help.
2. Immobilize the leg: Use a spl


###Observations
- The model demonstrates a clear understanding of the prompts and maintains relevant context throughout its responses.
- However, the answers tend to be overly generic and lack the depth or specificity necessary for practical use by medical professionals.
- Queries on more common or well-known medical topics often yield simpler, more generalized responses, limiting clinical utility.
- Across all tested prompts, the output length is consistently insufficient, suggesting that the current max_tokens setting is too low to allow comprehensive answers.

## Question Answering using LLM with Prompt Engineering

In [ ]:
def model_response(query, system_message, max_tokens,temperature,top_p,top_k):

    prompt = f"{user_prompt}\n{system_message}"

    model_output = lcpp_llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

###Prompt engineering & Parameter tuning Version 1

In [ ]:
max_tokens1 = 1100
temperature1 = 0
top_p1 = 0.95
top_k1 = 50
system_message1 = """
[INST]<<SYS>>
You are a licensed medical professional providing evidence-based, clinically accurate information. Given the medical scenario described, explain the following points that are relevant to the query:

Definition and overview of the condition or injury.
Common causes and risk factors relevant to the case.
Typical signs and symptoms, including any urgent “red-flag” indicators requiring immediate intervention.
Diagnostic approaches used to confirm the condition.
Evidence-based treatment options — include both initial/first-line management and definitive treatments, specifying when surgical, pharmacological, or other interventions are needed.
Recovery and prognosis, including follow-up care and preventive measures if applicable.

Use precise, professional medical terminology while keeping explanations clear enough for an educated layperson. When relevant, include considerations for resource-limited or emergency settings.
Structure the answer in a clear, numbered or bulleted format for easy reading.
<</SYS>>[/INST]
"""

###Prompt engineering & Parameter tuning Version 2

In [ ]:
# Reduce the max_tokens to reduce runtime, increase temperature for more natural language selection, system message is the same as version 1
max_tokens2 = 900
temperature2 = 0.1
top_p2 = 0.90
top_k2 = 30
system_message2 = """
[INST]<<SYS>>
You are a licensed medical professional providing evidence-based, clinically accurate information. Given the medical scenario described, explain the following points that are relevant to the query:

Definition and overview of the condition or injury.
Common causes and risk factors relevant to the case.
Typical signs and symptoms, including any urgent “red-flag” indicators requiring immediate intervention.
Diagnostic approaches used to confirm the condition.
Evidence-based treatment options — include both initial/first-line management and definitive treatments, specifying when surgical, pharmacological, or other interventions are needed.
Recovery and prognosis, including follow-up care and preventive measures if applicable.

Use precise, professional medical terminology while keeping explanations clear enough for an educated layperson. When relevant, include considerations for resource-limited or emergency settings.
Structure the answer in a clear, numbered or bulleted format for easy reading.
<</SYS>>[/INST]
"""

###Prompt engineering & Parameter tuning Version 3

In [ ]:
# The system prompt has been shortened and modified to ask for more context if needed and it is more specific about the kind of expected queries
max_tokens3 = 900
temperature3 = 0.1
top_p3 = 0.85
top_k3 = 30
system_message3 = """
[INST]<<SYS>>
Act as a medical knowledge specialist AI assistant. Your role is to provide detailed, evidence-based answers to questions about symptoms, diagnostics,
medications, treatment plans, and clinical protocols. Responses should be concise yet comprehensive, focusing strictly on the question’s context.
If the information given is unclear or insufficient, ask targeted follow-up questions to gather necessary details before providing a full answer.
Use bullet points or numbered steps for clarity, and highlight urgent or critical information when applicable.
<</SYS>>[/INST]
    """

###Prompt engineering & Parameter tuning Version 4

In [ ]:
#Different version of system prompt 3, reduced max_tokens and increased temperature and top_p.
max_tokens4 = 800
temperature4 = 0.2
top_p4 = 0.90
top_k4 = 30
system_message4 = """
    [INST]<<SYS>>
    You are an expert medical advisor. When answering medical queries involving diagnostics, drug information, treatment planning, specialty knowledge, or
    critical care protocols, provide accurate, relevant, and evidence-based responses. Tailor your level of detail according to the complexity of the question, aiming for clarity and precision.
    Format your responses flexibly using paragraphs, bullet points, or numbered lists as appropriate. If the query lacks context, ask clarifying questions
     before answering to ensure your advice is accurate and actionable.
    <</SYS>>[/INST]
    """

###Prompt engineering & Parameter tunning Version 5

In [ ]:
# Same prompt as in version 4, max_tokens increased to 1200, temperature reduced to 0.1
max_tokens5 = 1200
temperature5 = 0.1
top_p5 = 0.90
top_k5 = 30
system_message5 = """
    [INST]<<SYS>>
    You are an expert medical advisor. When answering medical queries involving diagnostics, drug information, treatment planning, specialty knowledge, or
    critical care protocols, provide accurate, relevant, and evidence-based responses. Tailor your level of detail according to the complexity of the question, aiming for clarity and precision.
    Format your responses flexibly using paragraphs, bullet points, or numbered lists as appropriate. If the query lacks context, ask clarifying questions
     before answering to ensure your advice is accurate and actionable.
    <</SYS>>[/INST]
    """

### Query 1: What is the protocol for managing sepsis in a critical care unit?

###Query 1, Response Version 1

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
prompt = f"{user_prompt}\n{system_message1}"
response = model_response(prompt, max_tokens=max_tokens1, temperature=temperature1, top_p=top_p1, top_k=top_k1)
print(response)

Llama.generate: prefix-match hit


1. Definition and Overview:
Sepsis is a life-threatening condition caused by a dysregulated host response to infection. The body's response to an infection, often bacterial, triggers a cascade of inflammatory responses that can lead to organ dysfunction or failure (septic shock). Severe sepsis and septic shock are the most extreme manifestations of sepsis.

2. Common Causes and Risk Factors:
Common causes include bacterial infections from various sources such as respiratory, urinary, gastrointestinal, or surgical sites. Risk factors include advanced age, chronic illnesses (e.g., diabetes, cancer), weakened immune system, intravascular devices, and recent surgery.

3. Signs and Symptoms:
Early signs may include fever, chills, rapid heart rate, rapid breathing, confusion, and increased white blood cell count. Severe sepsis can present with decreased urine output, altered mental status, lactic acidosis, and hypotension (low blood pressure). Red-flag symptoms requiring immediate interventi

###Query 1, Response Version 2

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
prompt = f"{user_prompt}\n{system_message2}"
response = model_response(prompt, max_tokens=max_tokens2, temperature=temperature2, top_p=top_p2, top_k=top_k2)
print(response)

Llama.generate: prefix-match hit


1. Definition and Overview:
Sepsis is a life-threatening condition caused by a dysregulated host response to infection. It can lead to tissue damage, organ failure, and even death if not recognized and treated promptly. Septic shock occurs when sepsis progresses to the point where vital organ functions are compromised.

2. Common Causes and Risk Factors:
Sepsis is most commonly caused by bacterial infections, particularly those affecting the lungs (pneumonia), urinary tract, or abdomen. Other risk factors include advanced age, chronic illnesses (e.g., diabetes, cancer), weakened immune systems, and invasive medical procedures (e.g., catheters).

3. Signs and Symptoms:
Early signs of sepsis may include fever, chills, rapid heart rate, increased respiratory rate, confusion, and decreased urine output. In severe cases, "red-flag" indicators such as low blood pressure, altered mental status, and decreased urine output require immediate intervention.

4. Diagnostic Approaches:
Diagnosis of 

###Query 1, Response Version 3

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
prompt = f"{user_prompt}\n{system_message3}"
response = model_response(prompt, max_tokens=max_tokens3, temperature=temperature3, top_p=top_p3, top_k=top_k3)
print(response)

Llama.generate: prefix-match hit


1. Recognition: Suspect sepsis in any patient with suspected or confirmed infection, and organ dysfunction using Sequential [Sepsis-related] Organ Failure Assessment (SOFA) score ≥2 points.
    2. Immediate resuscitation: Start fluid resuscitation to maintain adequate tissue perfusion (MAP >65 mmHg, CVP 8-12 mmHg, urine output >0.5 mL/kg/h).
         * Give 30 mL/kg crystalloids in the first hour and continue based on clinical response.
    3. Early antibiotic administration: Start broad-spectrum antibiotics within 1 hour of recognition if not already receiving them.
         * Adjust based on culture results and local guidelines.
    4. Source control: Identify and address the source of infection as soon as possible.
         * Surgical intervention may be required for abscesses, necrotizing infections, or other sources.
    5. Vasopressor support: Use vasopressors to maintain MAP >65 mmHg if fluid resuscitation alone is insufficient.
         * Start with a low-dose vasopressor (e.g.

###Query 1, Response Version 4

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
prompt = f"{user_prompt}\n{system_message4}"
response = model_response(prompt, max_tokens=max_tokens4, temperature=temperature4, top_p=top_p4, top_k=top_k4)
print(response)

Llama.generate: prefix-match hit


When managing sepsis in a critical care unit, prompt recognition and effective treatment are crucial for improving patient outcomes. Here's an evidence-based protocol for sepsis management:

1. **Suspect and recognize sepsis**: Assess patients with suspected infection for signs of sepsis, such as fever or hypothermia, tachycardia or bradycardia, respiratory distress, altered mental status, and lactic acidosis. Use the Sequential [Sepsis-related] Organ Failure Assessment (SOFA) score to evaluate organ dysfunction.

2. **Initiate early goal-directed therapy**: Begin resuscitation with intravenous fluids to maintain adequate blood pressure and urine output, aiming for a mean arterial pressure ≥65 mmHg and a central venous pressure (CVP) of 8-12 cm H2O. Consider administering vasopressors if needed to maintain MAP.

3. **Administration of broad-spectrum antibiotics**: Start empiric antimicrobial therapy as soon as possible based on the patient's clinical presentation, culture results, and 

###Query 1, Response Version 5

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
prompt = f"{user_prompt}\n{system_message5}"
response = model_response(prompt, max_tokens=max_tokens5, temperature=temperature5, top_p=top_p5, top_k=top_k5)
print(response)

Llama.generate: prefix-match hit


When managing sepsis in a critical care unit, prompt recognition and effective intervention are crucial for improving patient outcomes. Here's an evidence-based protocol for sepsis management:

1. **Suspect and recognize sepsis**: Assess patients with suspected infection for signs of sepsis, such as fever or hypothermia, tachycardia or bradycardia, respiratory distress, altered mental status, and lactic acidosis. Use the Sequential [Sepsis-related] Organ Failure Assessment (SOFA) score to evaluate organ dysfunction.

2. **Initiate early goal-directed therapy (EGDT)**: If sepsis is suspected, initiate EGDT as soon as possible. This includes:
   - Maintaining adequate mean arterial pressure (MAP) ≥65 mmHg or a systolic blood pressure of >100 mmHg to ensure adequate tissue perfusion.
   - Achieving a central venous oxygen saturation (ScvO2) ≥70% through fluid resuscitation and/or vasopressors.
   - Administering broad-spectrum antibiotics based on culture results or suspected infection so

####Observations:
- The highly structured prompt used in versions 1 and 2 constrained the model to follow a fixed output format, prioritizing adherence to the template over tailoring the content to the specific query. While this ensured consistent organization, it reduced flexibility and sometimes limited the relevance or adaptability of the response. For this reason, versions 1 and 2 will be discontinued for subsequent queries.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

###Query 2, Response Version 3

In [ ]:
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
prompt = f"{user_prompt}\n{system_message3}"
response = model_response(prompt, max_tokens=max_tokens3, temperature=temperature3, top_p=top_p3, top_k=top_k3)
print(response)

Llama.generate: prefix-match hit


 * Appendicitis is an inflammatory condition of the appendix, a small pouch that extends from the cecum (the beginning of the large intestine).
     * Common symptoms include:
         + Sudden pain in the lower right abdomen, which may begin as mild and gradually worsen over hours.
         + Loss of appetite.
         + Nausea and vomiting.
         + Fever (often low-grade at first).
         + Abdominal swelling and tenderness.
         + Pain that worsens when walking, coughing, or making other jarring movements.
     * Appendicitis cannot be cured via medicine alone. Antibiotics may help treat any accompanying infection but will not resolve the inflammation in the appendix.
     * The standard treatment for appendicitis is surgical removal of the appendix, known as an appendectomy. This procedure can be performed laparoscopically (using small incisions and a camera) or open (larger incision).
         + Laparoscopic appendectomy: Quicker recovery time, fewer complications, and le

###Query 2, Response Version 4

In [ ]:
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
prompt = f"{user_prompt}\n{system_message4}"
response = model_response(prompt, max_tokens=max_tokens4, temperature=temperature4, top_p=top_p4, top_k=top_k4)
print(response)

Llama.generate: prefix-match hit


Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The following are common symptoms for appendicitis:

    - Sudden pain that starts around the navel and then shifts to the lower right abdomen
    - Loss of appetite
    - Nausea and vomiting
    - Abdominal swelling
    - Fever, although not always present
    - Feeling bloated or gassy
    - Diarrhea or constipation
    - Inability to pass gas or have a bowel movement

If you suspect appendicitis based on these symptoms, it's essential to seek medical attention promptly. Appendicitis can be a serious condition if left untreated, as the appendix may rupture and cause peritonitis – an infection of the abdominal cavity that can lead to sepsis, organ damage, or even death.

There is no definitive cure for appendicitis with medication alone. Antibiotics might be prescribed to treat any potential infection, but they cannot resolve the 

###Query 2, Response Version 5

In [ ]:
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
prompt = f"{user_prompt}\n{system_message5}"
response = model_response(prompt, max_tokens=max_tokens5, temperature=temperature5, top_p=top_p5, top_k=top_k5)
print(response)

Llama.generate: prefix-match hit


Appendicitis is a medical condition characterized by inflammation of the appendix, a small pouch that extends from the cecum (the beginning of the large intestine). The following are common symptoms for appendicitis:

- Sudden pain in the lower right abdomen, which may begin as mild and gradually worsen over hours or days.
- Loss of appetite.
- Nausea and vomiting.
- Abdominal swelling and tenderness, especially in the lower right area.
- Fever and chills.
- Constipation or diarrhea.
- Pain that worsens when walking, coughing, or making other jarring movements.

Appendicitis is typically treated with surgery to remove the appendix before it ruptures and causes further complications. This procedure, called an appendectomy, can be performed as an open surgery or laparoscopically (using small incisions and a camera). The choice of surgical approach depends on factors such as the patient's overall health, the severity of the inflammation, and the surgeon's preference.

Medications alone ca

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

###Query 3, Response Version 3

In [ ]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
prompt = f"{user_prompt}\n{system_message3}"
response = model_response(prompt, max_tokens=max_tokens3, temperature=temperature3, top_p=top_p3, top_k=top_k3)
print(response)

Llama.generate: prefix-match hit


1. Sudden patchy hair loss, also known as alopecia areata, is an autoimmune disorder that causes hair loss in small, round patches on the scalp, face, or other areas of the body.
    2. Possible causes include genetics, stress, viral infections, and other autoimmune diseases. In some cases, no specific cause can be identified.
    3. Diagnosis is typically made based on physical examination and ruling out other conditions that may cause similar symptoms.
    4. Treatment options for alopecia areata include:
        * Topical treatments: Minoxidil (Rogaine) or corticosteroid creams or ointments can help stimulate hair growth.
        * Injections: Corticosteroids, such as triamcinolone acetonide or methotrexate, can be injected directly into the bald spots to promote regrowth.
        * Systemic treatments: Oral corticosteroids, immunosuppressive drugs, or biologics (such as infliximab or adalimumab) may be prescribed for more extensive hair loss or when other treatments are not effecti

###Query 3, Response Version 4

In [ ]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
prompt = f"{user_prompt}\n{system_message4}"
response = model_response(prompt, max_tokens=max_tokens4, temperature=temperature4, top_p=top_p4, top_k=top_k4)
print(response)

Llama.generate: prefix-match hit


I. Introduction

Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that results in the sudden onset of bald spots on the scalp or other areas of the body. The exact cause of this condition is not fully understood, but it's believed to be related to a combination of genetic and environmental factors. In some cases, stress or emotional trauma may trigger the onset of alopecia areata.

II. Diagnosis

The diagnosis of alopecia areata is typically made based on the appearance of the bald patches on the scalp. Your healthcare provider may perform a physical examination and ask about your medical history to rule out other possible causes of hair loss, such as thyroid disorders or nutritional deficiencies. In some cases, a biopsy may be performed to confirm the diagnosis.

III. Treatment Options

A. Topical treatments: Corticosteroids in the form of creams or ointments can be applied directly to the bald patches to help stimulate hair growth. Minoxidil (Ro

###Query 3, Response Version 5

In [ ]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
prompt = f"{user_prompt}\n{system_message5}"
response = model_response(prompt, max_tokens=max_tokens5, temperature=temperature5, top_p=top_p5, top_k=top_k5)
print(response)

Llama.generate: prefix-match hit


I. Introduction
        A. Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that results in the sudden appearance of bald spots on the scalp or other areas of the body.
        B. This condition can be distressing for individuals, but it's important to understand that it's not a sign of poor health or hygiene.
    
II. Possible Causes
        A. The exact cause of alopecia areata is unknown, but it's believed to be an autoimmune response where the body's immune system attacks hair follicles.
        B. Other potential triggers include stress, genetics, and viral infections.
    
III. Treatment Options
        1. Topical treatments: Minoxidil (Rogaine) or corticosteroid creams can be applied directly to the affected area to stimulate hair growth.
        2. Injections: Corticosteroids can be injected into the bald spots to reduce inflammation and promote hair regrowth.
        3. Systemic treatments: Oral corticosteroids, immunosuppressive drugs, o

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

###Query 4, Response Version 3

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
prompt = f"{user_prompt}\n{system_message3}"
response = model_response(prompt, max_tokens=max_tokens3, temperature=temperature3, top_p=top_p3, top_k=top_k3)
print(response)

Llama.generate: prefix-match hit


1. Traumatic Brain Injury (TBI) treatment depends on the severity and location of the injury. Here are general recommendations:
    2. _Immediate Care:_ Ensure airway is clear, provide oxygen if necessary, control bleeding, and prevent further injury.
    3. _Medical Evaluation:_ Hospitalization for observation, diagnostic tests (CT or MRI scan, EEG, etc.), and monitoring of vital signs.
    4. _Supportive Care:_ Provide adequate nutrition, hydration, and rest; manage symptoms such as pain, swelling, and seizures.
    5. _Rehabilitation:_ Physical therapy to improve motor skills, occupational therapy for cognitive and daily living skills, speech therapy for communication issues, and psychological counseling for emotional support.
    6. _Medications:_ Prescribed based on symptoms: anti-inflammatory drugs for swelling, pain relievers, anticonvulsants for seizures, and psychotropic medications for behavioral changes or depression.
    7. _Surgery:_ In some cases, surgery may be required 

###Query 4, Response Version 4

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
prompt = f"{user_prompt}\n{system_message4}"
response = model_response(prompt, max_tokens=max_tokens4, temperature=temperature4, top_p=top_p4, top_k=top_k4)
print(response)

Llama.generate: prefix-match hit


I. Introduction:
       A. Brain injuries can result in temporary or permanent impairment of brain function, leading to various symptoms such as headaches, memory loss, difficulty with speech or movement, and altered mood or behavior.
       B. The treatment for a brain injury depends on the severity and location of the injury, as well as the individual's overall health and age.
       C. In this response, we will discuss some common treatments for brain injuries, focusing on medical interventions and rehabilitation strategies.

    II. Medical Interventions:
       A. Acute Care: For severe brain injuries, initial treatment may include measures to prevent further damage, such as surgery to remove hematomas or decompressive craniectomy to relieve pressure within the skull.
         1. Medications: Patients may receive medications to manage symptoms, such as pain relievers, sedatives, and anti-seizure drugs.
         2. Intensive Care Unit (ICU) Monitoring: Brain injury patients often r

###Query 4, Response Version 5

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
prompt = f"{user_prompt}\n{system_message5}"
response = model_response(prompt, max_tokens=max_tokens5, temperature=temperature5, top_p=top_p5, top_k=top_k5)
print(response)

Llama.generate: prefix-match hit


I. Introduction:
       A. Brain injuries can result in a wide range of impairments, from mild concussions to severe traumatic brain injuries (TBI).
       B. Treatment for brain injuries depends on the severity and type of injury, as well as the individual's overall health and age.

 II. Immediate Care:
       A. For moderate to severe brain injuries, seek emergency medical attention right away.
          1. The primary goal is to prevent further damage and ensure the person receives appropriate care.
          2. Treatment may include: airway management, breathing support, fluid replacement, and medication to control intracranial pressure or seizures.

 III. Rehabilitation:
       A. Rehabilitation is an essential component of brain injury treatment, regardless of severity.
          1. The goal is to help the person regain lost skills and functions, improve overall health, and maximize independence.
          2. Rehabilitation may include physical therapy, occupational therapy, spee

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

###Query 5, Response Version 3

In [ ]:
user_prompt = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
prompt = f"{user_prompt}\n{system_message3}"
response = model_response(prompt, max_tokens=max_tokens3, temperature=temperature3, top_p=top_p3, top_k=top_k3)
print(response)

Llama.generate: prefix-match hit


1. Assess the severity of the injury: If you suspect a leg fracture during a hiking trip, first assess the patient's vital signs and overall condition. Check for signs of shock, such as pale skin, rapid heartbeat, or shallow breathing.
    2. Immobilize the injured leg: Use a splint, sling, or other immobilizing device to prevent any further movement of the fractured bone. Be careful not to apply too much pressure on the injury site.
    3. Provide pain relief: Administer over-the-counter pain medications like acetaminophen or ibuprofen as needed for pain management. Avoid giving aspirin, as it can interfere with blood clotting.
    4. Seek medical help: Arrange for transportation to a healthcare facility as soon as possible. A fracture requires proper diagnosis and treatment from a medical professional.
    5. Consider the type of fracture: Different types of leg fractures may require specific treatments, such as surgery or casting. Your healthcare provider will determine the best cou

###Query 5, Response Version 4

In [ ]:
user_prompt = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
prompt = f"{user_prompt}\n{system_message4}"
response = model_response(prompt, max_tokens=max_tokens4, temperature=temperature4, top_p=top_p4, top_k=top_k4)
print(response)

Llama.generate: prefix-match hit


I. Initial Assessment and Precautions for a Fractured Leg During Hiking:

   A. Safety: Ensure the safety of the injured person and any companions by securing the area if necessary.

   B. Immobilization: Splint or immobilize the leg above the fracture site using a makeshift splint, such as a stick or hiking pole, and clothing or padding to prevent further injury.

   C. Pain Management: Provide pain relief with over-the-counter pain medication like acetaminophen or ibuprofen if available. Be cautious about administering opioids due to potential side effects and addiction risks.

   D. Transportation: Arrange for transportation back to civilization, ideally via emergency medical services (EMS) if possible. If not, consider hiking out with assistance from companions or using a stretcher if necessary.

II. Treatment Steps upon Reaching Medical Care:

   A. Diagnostic Imaging: X-rays will be taken to confirm the fracture location and extent.

   B. Reduction: If the bone is displaced (out

###Query 5, Response Version 5

In [ ]:
user_prompt = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
prompt = f"{user_prompt}\n{system_message5}"
response = model_response(prompt, max_tokens=max_tokens5, temperature=temperature5, top_p=top_p5, top_k=top_k5)
print(response)

Llama.generate: prefix-match hit


I. Initial Assessment and Precautions for a Fractured Leg During Hiking:

   A. Evaluate the severity of the injury: Assess the degree of pain, swelling, deformity, and inability to bear weight on the affected leg.
      - If there are signs of an open fracture (skin penetration), severe bleeding, or nerve damage, call for emergency medical help immediately.

   B. Provide first aid: Apply a sterile dressing over any wounds, immobilize the leg using a splint or sling, and control bleeding if necessary.
      - Avoid moving the injured limb excessively to prevent further damage.

   C. Administer pain relief: Use over-the-counter pain medications like acetaminophen or ibuprofen as directed by label instructions.
      - Be cautious with narcotic pain relievers, as they may impair judgment and coordination needed for hiking safely.

II. Treatment and Recovery Considerations:

   A. Medical consultation: Seek professional medical advice to determine the type and extent of the fracture, wh

###Observations
- Versions 3, 4, and 5 return similar responses, addressing the query with a clear structure.
- Version 4, which uses a higher temperature value, shows noticeable changes in the terminology used for procedures. This results in lower readability.
- The lower maximum token limit in Version 3 (900) produces a more concise response, making it easier to read, but at the expense of omitting potentially relevant details.
- Considering these factors, Version 5 will be selected as the final version.

## Data Preparation for RAG

### Loading the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pdf_file = "/content/drive/MyDrive/PGP_AIML/NLP/medical_assistant/medical_diagnosis_manual.pdf"

In [ ]:
pdf_loader = PyPDFLoader(pdf_file)

In [ ]:
manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [ ]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(manual[i].page_content,end="\n")

Page Number : 1
ricrodriguezrj@gmail.com
WKIQY24LB5
This file is meant for personal use by ricrodriguezrj@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 2
ricrodriguezrj@gmail.com
WKIQY24LB5
This file is meant for personal use by ricrodriguezrj@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front  
  ................................................................................................................................................................................................................
1
Cover  
  .......................................................................................................................................................................................................
2
Front Matter  
  ..............................................................................................................

#### Checking the number of pages

In [ ]:
len(manual)

4114

### Data Chunking

###Chunking Version 1

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap= 50
)

In [ ]:
document_chunks = pdf_loader.load_and_split(text_splitter)

### Embedding

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

### Vector Database

In [ ]:
out_dir = 'medical_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [ ]:
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    persist_directory= "/content/drive/MyDrive/PGP_AIML/NLP/medical_assistant"
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
#Vector database saved in drive
vectorstore = Chroma(
    embedding_function=embedding_model,
    persist_directory="/content/drive/MyDrive/PGP_AIML/NLP/medical_assistant"
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


### Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3}
)

### System and User Prompt Template

In [ ]:
qna_system_message = """
    [INST]<<SYS>>
    You are an expert medical advisor. When answering medical queries involving diagnostics, drug information, treatment planning, specialty knowledge, or
    critical care protocols, provide accurate, relevant, and evidence-based responses. Tailor your level of detail according to the complexity of the question, aiming for clarity and precision.
    Format your responses flexibly using paragraphs, bullet points, or numbered lists as appropriate. If the query lacks context, ask clarifying questions
     before answering to ensure your advice is accurate and actionable.
    <</SYS>>[/INST]
    """

In [ ]:
qna_user_message_template = """
    Context: {context}
    Question: {question}
    Answer:
    """

### Response Function

In [ ]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = lcpp_llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
rag_response = generate_rag_response(user_prompt)
print(rag_response)

Llama.generate: prefix-match hit


In managing sepsis in a critical care unit, the following steps should be taken:
    
    1. Monitor vital signs and fluid status closely. Aim for CVP or PAOP of 8-12 mm Hg or 10 cm H2O respectively.
    2. Administer fluids to maintain adequate intravascular volume.
    3. Use vasopressors if necessary to maintain mean arterial pressure above 60 mmHg.
    4. Provide oxygen support as needed.
    5. Initiate broad-spectrum antibiotics based


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
rag_response = generate_rag_response(user_prompt)
print(rag_response)

Llama.generate: prefix-match hit


Common symptoms for appendicitis include:
    - Sudden pain in the lower right abdomen that begins near the navel and then shifts to your McBurney's point (right side, midway between the navel and the hip bone)
    - Loss of appetite
    - Nausea and vomiting
    - Abdominal swelling
    - Fever of 100.4°F (38°C) or higher
    - A feeling of not being able to pass gas or have a bowel movement
    
    Appendicitis cannot be


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
rag_response = generate_rag_response(user_prompt)
print(rag_response)

Llama.generate: prefix-match hit


Sudden patchy hair loss, also known as alopecia areata, is characterized by round or oval bald patches on the scalp. The exact cause of this condition is unknown but it's believed to be an autoimmune disorder that affects genetically susceptible individuals exposed to unclear environmental triggers.
    
    Treatment for alopecia areata includes:
    1. Topical corticosteroids: These medications can help reduce inflammation and promote hair regrowth. They come in various forms such as creams, ointments, or solutions.


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
rag_response = generate_rag_response(user_prompt)
print(rag_response)

Llama.generate: prefix-match hit


1. Traumatic brain injury (TBI) is characterized by physical damage to the brain that temporarily or permanently affects brain function.
    2. Initial treatment focuses on maintaining a reliable airway, adequate ventilation, oxygenation, and blood pressure. Surgery may be necessary for patients with more severe injuries to place monitors for intracranial pressure, decompress the brain, or remove hematomas.
    3. In the first few days after injury, ensuring adequate brain perfusion and oxygenation and preventing complications of altered consciousness are crucial.
    4. Subsequently, many patients require rehabilitation


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_prompt = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
rag_response = generate_rag_response(user_prompt)
print(rag_response)

Llama.generate: prefix-match hit


1. Assess the severity of the injury: Check for signs of open wounds, deformity, swelling, or inability to move the affected limb. If there are any signs of severe bleeding, nerve damage, or open fractures, seek immediate medical attention.
    2. Immobilize the leg: Use a splint or sling to prevent further injury and help alleviate pain. For more severe injuries, consider using a cast once the swelling has subsided.
    3. Administer first aid: Clean the wound if there is one, apply a sterile dressing, and cover it


### Fine-tuning

###Parameter tuning Version 1

In [ ]:
k1 = 3
max_tokens1 = 1000
temperature1 = 0
top_1p= 0.9
top_k1 = 30

###Parameter tuning Version 2

In [ ]:
k2 = 5
max_tokens2 = 1000
temperature2 = 0.2
top_2p= 0.9
top_k2 = 30

###Parameter tuning Version 3

In [ ]:
k3 = 4
max_tokens3 = 1000
temperature3 = 0.1
top_3p= 0.85
top_k3 = 30

###Parameter tuning Version 4

In [ ]:
k4 = 4
max_tokens4 = 1000
temperature4 = 0.1
top_4p= 0.85
top_k4 = 30

###Parameter tuning Version 5

In [ ]:
k5 = 4
max_tokens5 = 1200
temperature5 = 0.1
top_p5= 0.85
top_k5 = 30

Queries #1 and #4 will be considered for the evaluation of the parameter tuning versions to test the organization of the response based on the requirements of the query as well as the relevance of information in high risk/importance situations

###Query 1, Response Version 1

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
rag_response = generate_rag_response(user_prompt,k=k1,max_tokens=max_tokens1,temperature=temperature1,top_p=top_p1,top_k=top_k1)
print(rag_response)

Llama.generate: prefix-match hit


In managing sepsis in a critical care unit, the following steps should be taken:
    
    1. Monitor vital signs and fluid status closely. Aim for CVP or PAOP of 8-12 mm Hg or 10 cm H2O respectively.
    2. Administer fluids to maintain adequate intravascular volume.
    3. Use vasopressors if necessary to maintain mean arterial pressure above 60 mmHg.
    4. Provide oxygen support as needed.
    5. Initiate broad-spectrum antibiotics based on suspected source and organisms.
    6. Adjust antibiotic therapy based on culture and sensitivity results.
    7.


###Query 1, Response Version 2

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
rag_response = generate_rag_response(user_prompt,k=k2,max_tokens=max_tokens2,temperature=temperature2,top_p=top_p2,top_k=top_k2)
print(rag_response)

Llama.generate: prefix-match hit


In managing sepsis in a critical care unit, the following steps should be taken:
    
    1. Monitor vital signs, fluid intake and output, urine output, electrolyte levels, renal function, and blood glucose levels frequently.
    2. Provide adequate fluid resuscitation to maintain CVP or PAOP at target levels.
    3. Administer oxygen via mask or nasal prongs for respiratory support.
    4. Initiate vasopressors (dopamine, norepinephrine) if hypotension persists despite adequate fluid resuscitation.
    5. Begin empirical antibiotic therapy as soon as possible based on suspected source and sensitivity


###Query 1, Response Version 3

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
rag_response = generate_rag_response(user_prompt,k=k3,max_tokens=max_tokens3,temperature=temperature3,top_p=top_p3,top_k=top_k3)
print(rag_response)

Llama.generate: prefix-match hit


In managing sepsis in a critical care unit, the following steps should be taken:
    
    1. Monitor vital signs and fluid status closely. Aim for CVP or PAOP of 8-12 mm Hg or 10 cm H2O respectively.
    2. Administer fluids to maintain adequate intravascular volume. This may require large volumes (up to 10L over 4-12 hours).
    3. Provide oxygen support as needed. Intubation and mechanical ventilation may be required for respiratory failure.
    4. Initiate broad-spectrum antibiotics based on suspected source, clinical setting, and previous culture results.
         a. Empir


###Query 1, Response Version 4

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
rag_response = generate_rag_response(user_prompt,k=k4,max_tokens=max_tokens4,temperature=temperature4,top_p=top_p4,top_k=top_k4)
print(rag_response)

Llama.generate: prefix-match hit


In managing sepsis in a critical care unit, the following steps should be taken:
    
    1. Monitor vital signs and fluid status closely. Aim for CVP or PAOP of 8 mm Hg (10 cm H2O) or 12 to 15 mm Hg, respectively.
    2. Administer fluids to maintain adequate intravascular volume. This may require large volumes of crystalloid solution.
    3. Provide oxygen support as needed. Intubation and mechanical


###Query 1, Response Version 5

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
rag_response = generate_rag_response(user_prompt,k=k5,max_tokens=max_tokens5,temperature=temperature5,top_p=top_p5,top_k=top_k5)
print(rag_response)

Llama.generate: prefix-match hit


In managing sepsis in a critical care unit, the following steps should be taken:
    
    1. Monitor vital signs and fluid status closely. Aim to maintain CVP at 8 mm Hg (10 cm H2O) or PAOP at 12 to 15 mm Hg.
    2. Administer fluids aggressively to restore intravascular volume.
    3. Provide oxygen support as needed.
    4. Initiate vasopressors if the patient remains hypot


###Query 4, Response Version 1

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
rag_response = generate_rag_response(user_prompt,k=k1,max_tokens=max_tokens1,temperature=temperature1,top_p=top_p1,top_k=top_k1)
print(rag_response)

Llama.generate: prefix-match hit


1. Traumatic brain injury (TBI) is characterized by physical damage to the brain that temporarily or permanently affects brain function.
    2. Initial treatment focuses on maintaining a reliable airway, adequate ventilation, oxygenation, and blood pressure. Surgery may be necessary for patients with more severe injuries to place monitors for intracranial pressure, decompress the brain, or remove hematomas.
    3. In the first few days after injury, ensuring adequate brain perfusion and oxygenation and preventing complications of altered consciousness are crucial.
    4. Subsequently, many patients require rehabilitation to address functional impairments and prevent secondary disabilities.
    5. Rehabilitation involves a team approach that includes physical, occupational, and speech therapy, skill-building activities, and counseling to meet social and emotional needs.
    6. Brain injury support groups may provide assistance to families of brain-injured patients.
    7. For patients w

###Query 4, Response Version 2

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
rag_response = generate_rag_response(user_prompt,k=k2,max_tokens=max_tokens2,temperature=temperature2,top_p=top_p2,top_k=top_k2)
print(rag_response)

Llama.generate: prefix-match hit


1. Traumatic brain injury (TBI) is characterized by physical damage to the brain that temporarily or permanently affects brain function. Diagnosis is typically suspected clinically and confirmed through imaging, primarily CT scans.
    2. Initial treatment focuses on ensuring a reliable airway, maintaining adequate ventilation, oxygenation, and blood pressure. Surgery may be necessary for patients with more severe injuries to place monitors for intracranial pressure or to remove hematomas.
    3. During the acute phase following the injury, maintaining adequate brain perfusion and oxygenation is crucial, as well as preventing complications of altered consciousness.
    4. Subsequently, many TBI patients require rehabilitation to address functional impairments. Rehabilitation typically involves a team approach that combines physical, occupational, and speech therapy, skill-building activities, and counseling to meet the patient's social and emotional needs.
    5. Brain injury support g

###Query 4, Response Version 3

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
rag_response = generate_rag_response(user_prompt,k=k3,max_tokens=max_tokens3,temperature=temperature3,top_p=top_p3,top_k=top_k3)
print(rag_response)

Llama.generate: prefix-match hit


1. Traumatic brain injury (TBI) is characterized by physical damage to the brain that temporarily or permanently affects brain function.
    2. Initial treatment focuses on maintaining a reliable airway, adequate ventilation, oxygenation, and blood pressure. Surgery may be necessary for patients with more severe injuries to place monitors for intracranial pressure, decompress the brain, or remove hematomas.
    3. In the first few days after injury, ensuring adequate brain perfusion and oxygenation and preventing complications are crucial.
    4. Subsequently, many TBI patients require rehabilitation.
    5. Rehabilitation involves a team approach that includes physical, occupational, and speech therapy, skill-building activities, and counseling to address social and emotional needs.
    6. Brain injury support groups may also provide assistance to families of brain-injured patients.
    7. For patients whose coma lasts more than 24 hours, a prolonged period of rehabilitation, particul

###Query 4, Response Version 4

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
rag_response = generate_rag_response(user_prompt,k=k4,max_tokens=max_tokens4,temperature=temperature4,top_p=top_p4,top_k=top_k4)
print(rag_response)

Llama.generate: prefix-match hit


1. Traumatic Brain Injury (TBI) treatment includes ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure during the acute phase.
    2. Surgery may be needed to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas.
    3. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important.
    4. Subsequently, many patients require rehabilitation to regain lost functions and prevent secondary disabilities.
    5. Rehabilitation includes measures to prevent contractures, pressure ulcers, pneumonia, and other complications.
    6. The type and extent of rehabilitation therapy depend on the patient's abnormalities, which vary depending on the level and extent (partial or complete) of the injury.
    7. Complete transsection causes flaccid paralysis; partial transs

###Query 4, Response Version 5

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
rag_response = generate_rag_response(user_prompt,k=k5,max_tokens=max_tokens5,temperature=temperature5,top_p=top_p5,top_k=top_k5)
print(rag_response)

Llama.generate: prefix-match hit


1. Traumatic brain injury (TBI) is characterized by physical damage to the brain that temporarily or permanently affects brain function. Diagnosis is suspected clinically and confirmed through imaging, primarily CT scans.
    2. Initial treatment focuses on maintaining a reliable airway, adequate ventilation, oxygenation, and blood pressure. Surgery may be necessary for patients with more severe injuries to place monitors for intracranial pressure or remove hematomas.
    3. In the first few days after injury, ensuring adequate brain perfusion and oxygenation and preventing complications are crucial.
    4. Subsequently, many TBI patients require rehabilitation. Rehabilitation involves a team approach that combines physical, occupational, and speech therapy, skill-building activities, and counseling to meet the patient's social and emotional needs. Brain injury support groups may also provide assistance to families.
    5. For patients whose coma lasts more than 24 hours (approximately

###Observations

Versions 4 and 5 perform well for Query 4; however, the model struggles to generate a complete response for Query 1. To address this, increasing the retriever's k value may be necessary. Consequently, the LLM's context window will also need to be expanded to accommodate the additional retrieved information.

In [ ]:
llm1 = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=320,
    n_gpu_layers=24,
    n_ctx=2880
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


Updating the

In [ ]:
def generate_rag_response1(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm1(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

In [ ]:
k=4
max_tokens = 960
temperature = 0.1
top_p = 0.85
top_k = 30

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
rag_response1 = generate_rag_response1(user_prompt,k=k,max_tokens=max_tokens,temperature=temperature,top_p=top_p,top_k=top_k)
print(rag_response1)

Llama.generate: prefix-match hit


In managing sepsis in a critical care unit, the following steps should be taken:
    
    1. Fluid resuscitation: Administer 0.9% saline until central venous pressure (CVP) reaches 8 mm Hg or pulmonary artery occlusion pressure (PAOP) reaches 12 to 15 mm Hg. Oliguria with hypotension is not a contraindication for vigorous fluid resuscitation.
    2. Vasopressors: If the patient remains hypotensive after CVP or PAOP has been raised to target levels, dopamine may be given to increase mean arterial pressure (MAP) to at least 60 mm Hg. However, higher doses of dopamine and norepinephrine pose risks of organ hypoperfusion and acidosis.
    3. Oxygenation: Provide oxygen via mask or nasal prongs. Intubation and mechanical ventilation may be necessary for respiratory failure.
    4. Antibiotics: Administer broad-spectrum antibiotics based on suspected source, clinical setting, and previous culture results. Empiric therapy should be started immediately after suspecting sepsis.
    5. Source co

###Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
rag_response1 = generate_rag_response1(user_prompt,k=k,max_tokens=max_tokens,temperature=temperature,top_p=top_p,top_k=top_k)
print(rag_response1)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


1. Traumatic brain injury (TBI) is characterized by physical damage to the brain that temporarily or permanently affects its function. Diagnosis is typically suspected clinically and confirmed through imaging, primarily CT scans.
    2. Initial treatment focuses on maintaining a reliable airway, adequate ventilation, oxygenation, and blood pressure. Surgery may be necessary in more severe cases to place intracranial pressure monitors, decompress the brain, or remove hematomas.
    3. In the first few days following the injury, it is crucial to maintain adequate brain perfusion and oxygenation while preventing complications of altered consciousness.
    4. Subsequently, many patients require rehabilitation to address functional deficits and prevent secondary disabilities such as pressure ulcers, joint contractures, and pneumonia.
    5. Rehabilitation typically involves a team approach that combines physical, occupational, and speech therapy, skill-building activities, and counseling to

## Output Evaluation

In [ ]:
groundedness_rater_system_message  = """

You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

Please note: Make sure you give a single overall rating in the range of 1 to 5 along with an overall explanation.

"""

In [ ]:
relevance_rater_system_message = """

You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

Note: Provide a single overall rating in the range of 1 to 5, along with a brief explanation of why you assigned that score.
"""

In [ ]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [ ]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm1(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm1(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm1(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_prompt="What is the protocol for managing sepsis in a critical care unit?"
ground,rel = generate_ground_relevance_response(user_input=user_prompt,k=k,max_tokens=256,temperature=temperature,top_p=top_p,top_k=top_k)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 4 - The answer is derived from the context mostly. The context provides detailed information about the protocol for managing sepsis in a critical care unit, including monitoring requirements, fluid resuscitation, and the use of vasopressors. The answer summarizes these key points accurately. However, it does not include all the details provided in the context, such as the need for oxygen therapy and the importance of nutrition and infection prevention. Therefore, the answer is not a complete derivation from the context but mostly covers the main aspects of the protocol mentioned in the context.

 Rating: 5

Explanation: The context provides a detailed protocol for managing sepsis in a critical care unit, which is directly addressed by the answer. The answer covers key aspects of the question such as monitoring, fluid resuscitation, and the use of vasopressors. It also mentions some specific interventions like maintaining CVP and PAOP levels, and using dopamine if necessary. Th

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_prompt="What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
ground,rel = generate_ground_relevance_response(user_input=user_prompt,k=k,max_tokens=256,temperature=temperature,top_p=top_p,top_k=top_k)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 5 - The Answer is derived from the Context completely.

Explanation: The Question asks about the common symptoms for appendicitis and whether it can be cured via medicine. The Context provides detailed information on the diagnosis, treatment, and symptoms of appendicitis, including abdominal pain, loss of appetite, nausea and vomiting, fever, constipation or diarrhea, and the fact that appendicitis cannot be cured with medicine alone but requires surgical removal. The Answer directly quotes and summarizes this information, making it a clear derivation from the Context.

 Rating: 5

Explanation: The answer directly addresses all important aspects of the question by listing the common symptoms for appendicitis and stating that it cannot be cured with medicine alone, but rather requires surgical removal. The answer also mentions the standard treatment as an appendectomy and the possible use of antibiotics before surgery. The context provided in the text supports these statements,

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
user_prompt="What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
ground,rel = generate_ground_relevance_response(user_input=user_prompt,k=k,max_tokens=256,temperature=temperature,top_p=top_p,top_k=top_k)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 5

Explanation: The question asks about effective treatments and possible causes for sudden patchy hair loss, specifically alopecia areata. The context provided extensively discusses alopecia areata, its causes, and various treatment options including topical corticosteroids, intralesional corticosteroid injections, and topical immunotherapy. Therefore, the answer is completely derived from the context.

 Rating: 5

Explanation: The question asks about effective treatments and possible causes for sudden patchy hair loss, specifically alopecia areata. The context provided extensively covers the topic of alopecia areata, including its causes and various treatment options. The answer directly addresses the key aspects of the question by listing several effective treatments for alopecia areata, which aligns with the information presented in the context.


### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
user_prompt="What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
ground,rel = generate_ground_relevance_response(user_input=user_prompt,k=k,max_tokens=256,temperature=temperature,top_p=top_p,top_k=top_k)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 5 - The Answer is derived from the Context completely. The Context provides detailed information about the diagnosis, treatment, and rehabilitation for traumatic brain injury (TBI), including the recommended treatments mentioned in the Answer. These treatments are explicitly stated in the text, such as ensuring a reliable airway, maintaining adequate ventilation, oxygenation, and blood pressure, surgery to place monitors or decompress the brain, rehabilitation using physical, occupational, and speech therapy, skill-building activities, counseling, and family assistance from brain injury support groups. The Answer accurately reflects the information provided in the Context.

 Rating: 5

Explanation: The provided answer directly addresses all important aspects of the question by listing the recommended treatments for a person with a traumatic brain injury (TBI), including initial treatment, acute phase care, and rehabilitation. The context also mentions these treatments in detai

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
user_prompt="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
ground,rel = generate_ground_relevance_response(user_input=user_prompt,k=k,max_tokens=256,temperature=temperature,top_p=top_p,top_k=top_k)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 4 - The Answer is derived from the Context mostly.

Explanation: The user's question asks about the precautions and treatment steps for a person with a fractured leg during a hiking trip, as well as considerations for their care and recovery. The context provided in the text discusses various aspects of injuries, including fractures, symptoms, diagnosis, treatment options (such as immobilization, splinting, casting, and antibiotics), complications, and wound care.

The Answer covers several points from the Context, such as:
1. Immobilizing the affected limb using a splint or sling.
2. Seeking medical care promptly.
3. Monitoring for signs of infection or complications.
4. Administering pain relief.
5. Proper wound care.

These points are directly related to the information presented in the Context, making it a mostly derived Answer.

 Rating: 4

Explanation: The context provides detailed information about fractures, their symptoms, diagnosis, and treatment options, including i

- Overall, the model demonstrates strong performance, with responses that generally adhere closely to the context relevant to each query.
- A slight decrease in precision is observed when queries require step-by-step or list-style outputs. To address this, the temperature parameter will be set to 0 to encourage more deterministic and precise responses.

In [ ]:
user_prompt="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
ground,rel = generate_ground_relevance_response(user_input=user_prompt,k=k,max_tokens=256,temperature=0,top_p=top_p,top_k=top_k)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Rating: 4 - The Answer is derived from the Context mostly.

Explanation: The Question asks for precautions and treatment steps for a person with a fractured leg during a hiking trip, while the Context provides detailed information on various aspects of fracture care, including immobilization techniques (splints, casts), infection prevention, pain management, and complications associated with prolonged immobilization. The Answer directly addresses each point in the Question by referencing the relevant information from the Context.

 Rating: 5

Explanation: The AI-generated answer directly addresses all important aspects of the question, including necessary precautions (immobilization and seeking medical care), treatment steps (pain medication and elevating the limb), and potential complications (infection). The context provided in the text also supports these points.


## Actionable Insights and Business Recommendations

- While prompt engineering yielded seemingly detailed responses, this verbosity increased the risk of introducing unreliable information due to the generative nature of LLMs. The Retrieval-Augmented Generation (RAG) approach, in contrast, demonstrated consistently high groundness and relevance scores (4–5 out of 5) across all tested queries. From a safety standpoint, the RAG model is the preferred choice, as it delivers accuracy and factual reliability—both critical for the medical domain.
- To further improve the relevance and completeness of responses, it is recommended to curate specialized, detail-rich documents for complex or information-heavy sections of the medical diagnosis manual. This will enhance retrieval quality and ensure the model has authoritative material for nuanced medical topics.
- While the model performs well on current test queries, coverage could be further enhanced by incorporating a broader range of medical references (e.g., drug formularies, treatment protocols from multiple health organizations, and peer-reviewed studies). This would make the model more robust in handling edge cases and specialized questions.
- Although RAG outperformed prompt engineering overall, a hybrid approach could be explored for non-critical, patient-friendly explanations where richer contextualization and softer language may be beneficial. In these cases, prompt tuning on top of RAG output might increase user satisfaction without compromising accuracy.
- A slight reduction in precision may occur when queries request step-by-step explanations or list-style outputs. While the model still provides relevant responses, performance improves when these queries are submitted individually without additional secondary questions.
- The LLM used in this evaluation has been adapted for lower computing resource requirements. For scenarios where handling larger volumes of information, achieving higher accuracy, or producing richer and more detailed responses is a priority, a larger version of the model is recommended. The runtime and responsiveness will vary depending on the hardware used to submit queries and process responses.